In [1]:
!pip install sqlalchemy

In [2]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback 
import glob
import os 
import pprint
from pprint import pprint
import json
import requests
import time
from IPython.display import display

In [3]:
url = "dbbikes.cvvkn6jkfbdf.eu-west-1.rds.amazonaws.com"
port = "3306"
db = "dbbikes"
user = "SEGroup25"
password = "SEGroup25?"

In [4]:
engine = create_engine(f"mysql+mysqldb://{user}:{password}@{url}:{port}/{db}", echo=True)

In [5]:
for res in engine.execute("SHOW VARIABLES"):
    print(res)

2023-03-02 21:49:38,545 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-03-02 21:49:38,546 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-02 21:49:38,583 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-03-02 21:49:38,584 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-02 21:49:38,601 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-03-02 21:49:38,601 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-02 21:49:38,632 INFO sqlalchemy.engine.Engine SHOW VARIABLES
2023-03-02 21:49:38,633 INFO sqlalchemy.engine.Engine [raw sql] ()
('activate_all_roles_on_login', 'OFF')
('admin_address', '')
('admin_port', '33062')
('admin_ssl_ca', '')
('admin_ssl_capath', '')
('admin_ssl_cert', '')
('admin_ssl_cipher', '')
('admin_ssl_crl', '')
('admin_ssl_crlpath', '')
('admin_ssl_key', '')
('admin_tls_ciphersuites', '')
('admin_tls_version', 'TLSv1.2,TLSv1.3')
('authentication_policy', '*')
('auto_generate_certs', 'ON')
('auto_increment_increment', '1')
('auto_increment_

('innodb_stats_persistent_sample_pages', '20')
('innodb_stats_transient_sample_pages', '8')
('innodb_status_output', 'OFF')
('innodb_status_output_locks', 'OFF')
('innodb_strict_mode', 'ON')
('innodb_sync_array_size', '1')
('innodb_sync_spin_loops', '30')
('innodb_table_locks', 'ON')
('innodb_temp_data_file_path', 'ibtmp1:12M:autoextend')
('innodb_temp_tablespaces_dir', './#innodb_temp/')
('innodb_thread_concurrency', '0')
('innodb_thread_sleep_delay', '10000')
('innodb_tmpdir', '')
('innodb_undo_directory', './')
('innodb_undo_log_encrypt', 'OFF')
('innodb_undo_log_truncate', 'ON')
('innodb_undo_tablespaces', '2')
('innodb_use_fdatasync', 'OFF')
('innodb_use_native_aio', 'ON')
('innodb_validate_tablespace_paths', 'ON')
('innodb_version', '8.0.28')
('innodb_write_io_threads', '4')
('insert_id', '0')
('interactive_timeout', '28800')
('internal_tmp_mem_storage_engine', 'TempTable')
('join_buffer_size', '262144')
('keep_files_on_create', 'OFF')
('key_buffer_size', '16777216')
('key_cache_

In [6]:
sql = """
CREATE TABLE IF NOT EXISTS station (
contract_name VARCHAR(15),
name VARCHAR(365),
number INT,
address VARCHAR(365),
position_lat REAL,
position_long REAL,
banking INT,
bonus INT,
bike_stands INT,
status VARCHAR(50)
)
"""

try:
    res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-03-02 21:49:38,830 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS station
2023-03-02 21:49:38,832 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-02 21:49:38,863 INFO sqlalchemy.engine.Engine COMMIT
2023-03-02 21:49:38,894 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS station (
contract_name VARCHAR(15),
name VARCHAR(365),
number INT,
address VARCHAR(365),
position_lat REAL,
position_long REAL,
banking INT,
bonus INT,
bike_stands INT,
status VARCHAR(50)
)

2023-03-02 21:49:38,894 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-02 21:49:38,937 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [7]:
sql = """
CREATE TABLE IF NOT EXISTS availability (
available_bike_stands INT,
available_bikes INT,
last_update INT,
number INT)
"""

try:
    res = engine.execute(sql)
    print(res.fetchall())
except Exception as e:
    print(e)

2023-03-02 21:49:38,979 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS availability (
available_bike_stands INT,
available_bikes INT,
last_update INT,
number INT)

2023-03-02 21:49:38,981 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-02 21:49:39,002 INFO sqlalchemy.engine.Engine COMMIT
This result object does not return rows. It has been closed automatically.


In [8]:
import requests
import traceback
import datetime
import time 

In [9]:
APIKEY = "5ac1b590c7e338d66c34fa0c5610e94c99a3b45c"
NAME = "Dublin"
STATIONS = "https://api.jcdecaux.com/vls/v1/stations"

In [ ]:
r = requests.get(STATIONS , params = {"apiKey": APIKEY, "contract": NAME})
def write_to_file(text):
    now = datetime.datetime.now()
    with open(f"data/bikes_{now}".replace(" ", "_"), "w") as f:
        f.write(r.text)
    

            
def stations_to_db(text):
    stations = json.loads(text)
    print(type(stations),len(stations))
    for station in stations:
        print(station)
        vals = (
            station.get("contract_name"),
            station.get("name"),
            station.get("number"),
            station.get("address"),
            station.get("position").get("lat"),
            station.get("position").get("long"),
            int(station.get("banking")),
            int(station.get("bonus")),
            station.get("bike_stands"),
            station.get("status")
        )

        engine.execute("insert into station values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", vals)
        
    return
   

    stations_to_db(r.text)

def availability_to_db(text):
    availability =json.loads(text)
    print(type(availability), len(availability))
    for available in availability:
        print(available)
        vals = (
        (available.get("available_bike_stands")),
        (available.get("available_bikes")),
        available.get("last_update"),
        (available.get("number"))
        )
        engine.execute("insert into availability values (%s, %s, %s, %s)", vals)
    return


    availability_to_db(r.text)

def main():
    while True:
        try:
            now = datetime.datetime.now()
            r = requests.get(STATIONS , params = {"apiKey": APIKEY, "contract": NAME})
            print(r,now)
            write_to_file(r.text)
            stations_to_db(r.text)
            availability_to_db(r.text)
            time.sleep(5 * 60)
        except:
            print(traceback.format_exc())
            if engine is None:
                return

if __name__ == "__main__":
    main()  

<Response [200]> 2023-03-02 21:49:39.513134
<class 'list'> 115
{'number': 42, 'contract_name': 'dublin', 'name': 'SMITHFIELD NORTH', 'address': 'Smithfield North', 'position': {'lat': 53.349562, 'lng': -6.278198}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 0, 'available_bikes': 30, 'status': 'OPEN', 'last_update': 1677793474000}
2023-03-02 21:49:39,972 INFO sqlalchemy.engine.Engine insert into station values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-02 21:49:39,973 INFO sqlalchemy.engine.Engine [raw sql] ('dublin', 'SMITHFIELD NORTH', 42, 'Smithfield North', 53.349562, None, 0, 0, 30, 'OPEN')
2023-03-02 21:49:39,991 INFO sqlalchemy.engine.Engine COMMIT
{'number': 30, 'contract_name': 'dublin', 'name': 'PARNELL SQUARE NORTH', 'address': 'Parnell Square North', 'position': {'lat': 53.3537415547453, 'lng': -6.26530144781526}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 15, 'available_bikes': 5, 'status': 'OPEN', '

2023-03-02 21:49:41,252 INFO sqlalchemy.engine.Engine [raw sql] ('dublin', 'PORTOBELLO ROAD', 43, 'Portobello Road', 53.330091, None, 0, 0, 30, 'OPEN')
2023-03-02 21:49:41,266 INFO sqlalchemy.engine.Engine COMMIT
{'number': 31, 'contract_name': 'dublin', 'name': 'PARNELL STREET', 'address': 'Parnell Street', 'position': {'lat': 53.350929, 'lng': -6.265125}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 12, 'available_bikes': 8, 'status': 'OPEN', 'last_update': 1677793233000}
2023-03-02 21:49:41,302 INFO sqlalchemy.engine.Engine insert into station values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-02 21:49:41,303 INFO sqlalchemy.engine.Engine [raw sql] ('dublin', 'PARNELL STREET', 31, 'Parnell Street', 53.350929, None, 0, 0, 20, 'OPEN')
2023-03-02 21:49:41,322 INFO sqlalchemy.engine.Engine COMMIT
{'number': 98, 'contract_name': 'dublin', 'name': 'FREDERICK STREET SOUTH', 'address': 'Frederick Street South', 'position': {'lat': 53.341515, 'lng': -6.2

2023-03-02 21:49:42,058 INFO sqlalchemy.engine.Engine [raw sql] ('dublin', 'EARLSFORT TERRACE', 11, 'Earlsfort Terrace', 53.334295, None, 0, 0, 30, 'OPEN')
2023-03-02 21:49:42,077 INFO sqlalchemy.engine.Engine COMMIT
{'number': 17, 'contract_name': 'dublin', 'name': 'GOLDEN LANE', 'address': 'Golden Lane', 'position': {'lat': 53.340803, 'lng': -6.267732}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 18, 'available_bikes': 2, 'status': 'OPEN', 'last_update': 1677793420000}
2023-03-02 21:49:42,121 INFO sqlalchemy.engine.Engine insert into station values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-02 21:49:42,123 INFO sqlalchemy.engine.Engine [raw sql] ('dublin', 'GOLDEN LANE', 17, 'Golden Lane', 53.340803, None, 0, 0, 20, 'OPEN')
2023-03-02 21:49:42,144 INFO sqlalchemy.engine.Engine COMMIT
{'number': 45, 'contract_name': 'dublin', 'name': 'DEVERELL PLACE', 'address': 'Deverell Place', 'position': {'lat': 53.351464, 'lng': -6.255265}, 'banking': False

2023-03-02 21:49:42,810 INFO sqlalchemy.engine.Engine [raw sql] ('dublin', 'LIME STREET', 62, 'Lime Street', 53.346026, None, 0, 0, 40, 'OPEN')
2023-03-02 21:49:42,829 INFO sqlalchemy.engine.Engine COMMIT
{'number': 5, 'contract_name': 'dublin', 'name': 'CHARLEMONT PLACE', 'address': 'Charlemont Street', 'position': {'lat': 53.330662, 'lng': -6.260177}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 0, 'available_bikes': 39, 'status': 'OPEN', 'last_update': 1677793643000}
2023-03-02 21:49:42,863 INFO sqlalchemy.engine.Engine insert into station values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-02 21:49:42,865 INFO sqlalchemy.engine.Engine [raw sql] ('dublin', 'CHARLEMONT PLACE', 5, 'Charlemont Street', 53.330662, None, 0, 0, 40, 'OPEN')
2023-03-02 21:49:42,881 INFO sqlalchemy.engine.Engine COMMIT
{'number': 97, 'contract_name': 'dublin', 'name': 'KILMAINHAM GAOL', 'address': 'Kilmainham Gaol', 'position': {'lat': 53.342113, 'lng': -6.310015}, 'banki

2023-03-02 21:49:43,594 INFO sqlalchemy.engine.Engine [raw sql] ('dublin', 'NEWMAN HOUSE', 53, 'Newman House', 53.337132, None, 0, 0, 40, 'OPEN')
2023-03-02 21:49:43,612 INFO sqlalchemy.engine.Engine COMMIT
{'number': 58, 'contract_name': 'dublin', 'name': "SIR PATRICK DUN'S", 'address': "Sir Patrick's Dun", 'position': {'lat': 53.339218, 'lng': -6.240642}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 38, 'available_bikes': 2, 'status': 'OPEN', 'last_update': 1677793347000}
2023-03-02 21:49:43,652 INFO sqlalchemy.engine.Engine insert into station values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
2023-03-02 21:49:43,653 INFO sqlalchemy.engine.Engine [raw sql] ('dublin', "SIR PATRICK DUN'S", 58, "Sir Patrick's Dun", 53.339218, None, 0, 0, 40, 'OPEN')
2023-03-02 21:49:43,668 INFO sqlalchemy.engine.Engine COMMIT
{'number': 66, 'contract_name': 'dublin', 'name': 'NEW CENTRAL BANK', 'address': 'New Central Bank', 'position': {'lat': 53.347122, 'lng': -6.234749}

...

2023-03-02 21:49:50,228 INFO sqlalchemy.engine.Engine [raw sql] (30, 0, 1677793317000, 19)
2023-03-02 21:49:50,243 INFO sqlalchemy.engine.Engine COMMIT
{'number': 7, 'contract_name': 'dublin', 'name': 'HIGH STREET', 'address': 'High Street', 'position': {'lat': 53.343565, 'lng': -6.275071}, 'banking': False, 'bonus': False, 'bike_stands': 29, 'available_bike_stands': 7, 'available_bikes': 22, 'status': 'OPEN', 'last_update': 1677793550000}
2023-03-02 21:49:50,292 INFO sqlalchemy.engine.Engine insert into availability values (%s, %s, %s, %s)
2023-03-02 21:49:50,294 INFO sqlalchemy.engine.Engine [raw sql] (7, 22, 1677793550000, 7)
2023-03-02 21:49:50,311 INFO sqlalchemy.engine.Engine COMMIT
{'number': 60, 'contract_name': 'dublin', 'name': 'NORTH CIRCULAR ROAD', 'address': 'North Circular Road', 'position': {'lat': 53.359624, 'lng': -6.260348}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 8, 'available_bikes': 22, 'status': 'OPEN', 'last_update': 16777936

2023-03-02 21:49:51,146 INFO sqlalchemy.engine.Engine [raw sql] (7, 13, 1677793274000, 16)
2023-03-02 21:49:51,163 INFO sqlalchemy.engine.Engine COMMIT
{'number': 96, 'contract_name': 'dublin', 'name': 'KILMAINHAM LANE', 'address': 'Kilmainham Lane', 'position': {'lat': 53.341805, 'lng': -6.305085}, 'banking': False, 'bonus': False, 'bike_stands': 30, 'available_bike_stands': 19, 'available_bikes': 11, 'status': 'OPEN', 'last_update': 1677793369000}
2023-03-02 21:49:51,198 INFO sqlalchemy.engine.Engine insert into availability values (%s, %s, %s, %s)
2023-03-02 21:49:51,200 INFO sqlalchemy.engine.Engine [raw sql] (19, 11, 1677793369000, 96)
2023-03-02 21:49:51,217 INFO sqlalchemy.engine.Engine COMMIT
{'number': 82, 'contract_name': 'dublin', 'name': 'MOUNT BROWN', 'address': 'Mount Brown', 'position': {'lat': 53.341645, 'lng': -6.29719}, 'banking': False, 'bonus': False, 'bike_stands': 22, 'available_bike_stands': 0, 'available_bikes': 8, 'status': 'OPEN', 'last_update': 1677793221000}

2023-03-02 21:49:52,096 INFO sqlalchemy.engine.Engine [raw sql] (0, 39, 1677793182000, 93)
2023-03-02 21:49:52,112 INFO sqlalchemy.engine.Engine COMMIT
{'number': 22, 'contract_name': 'dublin', 'name': 'TOWNSEND STREET', 'address': 'Townsend Street', 'position': {'lat': 53.345922, 'lng': -6.254614}, 'banking': False, 'bonus': False, 'bike_stands': 20, 'available_bike_stands': 11, 'available_bikes': 8, 'status': 'OPEN', 'last_update': 1677793608000}
2023-03-02 21:49:52,142 INFO sqlalchemy.engine.Engine insert into availability values (%s, %s, %s, %s)
2023-03-02 21:49:52,144 INFO sqlalchemy.engine.Engine [raw sql] (11, 8, 1677793608000, 22)
2023-03-02 21:49:52,202 INFO sqlalchemy.engine.Engine COMMIT
{'number': 50, 'contract_name': 'dublin', 'name': 'GEORGES LANE', 'address': "George's Lane", 'position': {'lat': 53.35023, 'lng': -6.279696}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 20, 'available_bikes': 20, 'status': 'OPEN', 'last_update': 16777936830

2023-03-02 21:49:54,164 INFO sqlalchemy.engine.Engine [raw sql] (8, 20, 1677793402000, 35)
2023-03-02 21:49:54,183 INFO sqlalchemy.engine.Engine COMMIT
{'number': 10, 'contract_name': 'dublin', 'name': 'DAME STREET', 'address': 'Dame Street', 'position': {'lat': 53.344007, 'lng': -6.266802}, 'banking': True, 'bonus': False, 'bike_stands': 16, 'available_bike_stands': 2, 'available_bikes': 14, 'status': 'OPEN', 'last_update': 1677793606000}
2023-03-02 21:49:54,221 INFO sqlalchemy.engine.Engine insert into availability values (%s, %s, %s, %s)
2023-03-02 21:49:54,222 INFO sqlalchemy.engine.Engine [raw sql] (2, 14, 1677793606000, 10)
2023-03-02 21:49:54,240 INFO sqlalchemy.engine.Engine COMMIT
{'number': 100, 'contract_name': 'dublin', 'name': 'HEUSTON BRIDGE (SOUTH)', 'address': 'Heuston Bridge (South)', 'position': {'lat': 53.347106, 'lng': -6.292041}, 'banking': False, 'bonus': False, 'bike_stands': 25, 'available_bike_stands': 0, 'available_bikes': 25, 'status': 'OPEN', 'last_update': 

2023-03-02 21:49:55,783 INFO sqlalchemy.engine.Engine [raw sql] (15, 6, 1677793565000, 40)
2023-03-02 21:49:55,798 INFO sqlalchemy.engine.Engine COMMIT
{'number': 29, 'contract_name': 'dublin', 'name': 'ORMOND QUAY UPPER', 'address': 'Ormond Quay Upper', 'position': {'lat': 53.346057, 'lng': -6.268001}, 'banking': False, 'bonus': False, 'bike_stands': 29, 'available_bike_stands': 21, 'available_bikes': 8, 'status': 'OPEN', 'last_update': 1677793421000}
2023-03-02 21:49:55,977 INFO sqlalchemy.engine.Engine insert into availability values (%s, %s, %s, %s)
2023-03-02 21:49:55,979 INFO sqlalchemy.engine.Engine [raw sql] (21, 8, 1677793421000, 29)
2023-03-02 21:49:56,132 INFO sqlalchemy.engine.Engine COMMIT
{'number': 103, 'contract_name': 'dublin', 'name': 'GRANGEGORMAN LOWER (SOUTH)', 'address': 'Grangegorman Lower (South)', 'position': {'lat': 53.354663, 'lng': -6.278681}, 'banking': False, 'bonus': False, 'bike_stands': 40, 'available_bike_stands': 37, 'available_bikes': 3, 'status': 'O

In [ ]:
#number, name, address, latitude, longitude, bike_stands